In [55]:
import bkutils
import warnings
import os
import pandas as pd
from tqdm.notebook import tqdm
warnings.filterwarnings(action="ignore")
os.getcwd()

seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)

bkstation_info = pd.read_csv(
    "assets/bkstation_info.csv", encoding="CP949", index_col=0
)

In [2]:
bkstation_info.head()

,value,label,coor,num
0,102,망원역 1번출구 앞,"[37.555649, 126.910629]",62074
1,103,망원역 2번출구 앞,"[37.554951, 126.910835]",51577
2,104,합정역 1번출구 앞,"[37.550629, 126.914986]",25702
3,105,합정역 5번출구 앞,"[37.550007, 126.914825]",20609
4,106,합정역 7번출구 앞,"[37.548645, 126.912827]",63341


In [56]:
station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)



In [57]:
a = station[['latitude','longtitude']].values.tolist()

for i,var in enumerate(a[:2]) : 
    bkstation_info.at[i,'coor'] =var

In [59]:
bkstation_info.to_csv('bkstation_info.csv', encoding='CP949')

In [2]:
import numpy as np


def haversine_np(lon1: float, lat1: float, lon2: float, lat2: float
) -> np.array:
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    )

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m

In [10]:
from typing import Dict
# Dict[int,str,list,int,float,float]
def arrStation(st_id : int) :
    # 대여소 추출기록
    filtered_data = raw_data(st_id)
    st_id1_record = filtered_data.groupby(by='st_id1').size()
    st_id2_record = filtered_data.groupby(by='st_id2').size()

    all_record = st_id1_record.add(st_id2_record, fill_value=0)

    # 대여기록 100건 이상 총 200개 대여소만 추출하기
    sort_info = all_record[all_record>50].sort_values(ascending=False)[1:201] 
    station_id = sort_info.index.to_list()
    
    result = bkstation_info.query('value == @station_id').reset_index(drop=True) 
    # return result

    result_station = []
    for j in result['value'] :
        BM = (filtered_data["st_id2"] == j )
        st_id1_time = (
            filtered_data[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        BM = filtered_data["st_id1"] == j
        st_id2_time = (
            filtered_data[BM]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)
        )
        ### 1to2 걸린시간 + 2to1 걸린시간 합
        all_rent = st_id1_time.add(st_id2_time, fill_value=0)

        ### 1to2 & 2to1 대여기록 합
        total_record = all_rent.sum()

        k = []
        i = 2
        ### 기록 많은 순만 종합
        while len(k) < 1:
            k = all_rent[all_rent >= (total_record / i)]
            i = i * 1.5

        ### 대여시간
        ind = k.index
        ### 대여기록
        val = k.values
        ### 대여기록 합
        a = k.sum()
        ### 대여시간 * 대여기록
        asddd = sum([a * b for a, b in zip(ind, val)])
        # 평균 시간
        ddddd = asddd / a

        # 올림
        val = round(ddddd, 0)

        ### 평균거리 계산
        BM = (filtered_data["st_id2"] == j )
        st_id1_time = (
            filtered_data[BM]["dist"]
            .value_counts()
            .sort_values(ascending=False)
        )
        BM = filtered_data["st_id1"] == j
        st_id2_time = (
            filtered_data[BM]["dist"]
            .value_counts()
            .sort_values(ascending=False)
        )
        all_rent = st_id1_time.add(st_id2_time, fill_value=0)
        dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
        num=3
        vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
        avg_dist = sum(vals)/sum(dist.to_list()[:num])


        result_station.append([val,total_record,(round(avg_dist/1000,2))])

    df = pd.DataFrame(result_station)
    finish = pd.concat([result, df],axis=1)
    finish.columns = ['value','label','coor','num','time','record','dist']
    # result['dist'] = avg_dist_list
    return finish # min과 max를 설정할 수 있도록 순번 잊지말기 

abb = arrStation(272)


In [24]:
dep = bkstation_info.iloc[[3]]
arr = bkstation_info.iloc[[11]]
route_coor(pd.Series([]),arr)



TypeError: DataFrame만 가능 현재 : <class 'pandas.core.series.Series'>

In [21]:
import json
import requests
import time
import ast
def route_coor(
        departure_station,
        arrival_station,):
    # ) -> List :
        """departure_station과 arrival_station은 모두 dataframe으로!"""
        if isinstance(departure_station, pd.Series):
            raise TypeError(f'DataFrame 가능... 현재 Type : {type(departure_station)}')

        if isinstance(arrival_station, pd.Series):
            raise TypeError(f'DataFrame 가능... 현재 Type {type(arrival_station)}')
        if departure_station.empty or arrival_station.empty:
            raise ValueError('Value is Empty!')

        departure = ast.literal_eval(departure_station['coor'].values[0])
        arrival = ast.literal_eval(arrival_station['coor'].values[0])

        for i in range(3):
            url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

            payload = {
                "angle": 0,
                "speed": 0,
                "reqCoordType": "WGS84GEO",
                "searchOption": "0",
                "resCoordType": "WGS84GEO",
                "sort": "index",
                "startX": departure[1],
                "startY": departure[0],
                "endX": arrival[1],
                "endY": arrival[0],
                "startName": "출발",
                "endName": "도착",
            }
            headers = {
                "Accept": "application/json",
                "Content-Type": "application/json",
                "appKey": "l7xxfdc75c1509a74ecdba02bf5e024ee9d5",
            }

            response = requests.post(url, json=payload, headers=headers).text

            api_data = json.loads(response).get("features")
            if api_data != None:
                time.sleep(0.3)
                break
            print("!!!!!!!!!!!!!", api_data, f"{i+1} 회 시도중")

        coor_raw_data = [
            api_data[i].get("geometry").get("coordinates")
            for i in range(0, len(api_data))
        ]

        finish_data = []
        for data in coor_raw_data:
            # data = api_data[1].get('geometry').get('coordinates')
            if type(data[0]) == list:
                for i in data:
                    finish_data.append(i)
            else:
                pass
        return pd.DataFrame(finish_data)[[1, 0]].values  ## 위도 경도 위치 바꾸기

In [ ]:
for j in abb['value'][:1] :
    ### 평균거리 계산
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)
    dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
    num=3
    vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
    avg_dist = sum(vals)/sum(dist.to_list()[:num])
    print(dist)
    print(j,avg_dist)

In [ ]:
abb

In [231]:
result_station = []
for j in abb['value'] :
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["riding_time"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["riding_time"]
        .value_counts()
        .sort_values(ascending=False)
    )
    ### 1to2 걸린시간 + 2to1 걸린시간 합
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)

    ### 1to2 & 2to1 대여기록 합
    total_record = all_rent.sum()

    k = []
    i = 2
    ### 기록 많은 순만 종합
    while len(k) < 1:
        k = all_rent[all_rent >= (total_record / i)]
        i = i * 1.5

    ### 대여시간
    ind = k.index
    ### 대여기록
    val = k.values
    ### 대여기록 합
    a = k.sum()
    ### 대여시간 * 대여기록
    asddd = sum([a * b for a, b in zip(ind, val)])
    # 평균 시간
    ddddd = asddd / a

    # 올림
    result = round(ddddd, 0)

    ### 평균거리 계산
    BM = (filtered_data["st_id2"] == j )
    st_id1_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    BM = filtered_data["st_id1"] == j
    st_id2_time = (
        filtered_data[BM]["dist"]
        .value_counts()
        .sort_values(ascending=False)
    )
    all_rent = st_id1_time.add(st_id2_time, fill_value=0)
    dist = pd.cut(all_rent.index,bins=50).value_counts().sort_values(ascending=False)
    num=3
    vals = [dist[:num].index[a].mid * dist.iloc[a] for a in range(3)]
    avg_dist = sum(vals)/sum(dist.to_list()[:num])


    result_station.append([result,total_record,avg_dist])


